# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [ ]:
import os
os.environ["ANTHROPIC_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../.env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [2]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("pirate_man", include_model=False)  # Changed to False

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [13]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate_man', 'lc_hub_commit_hash': 'ab49f4279842352da4ecf9217b015ceae60e9aea06b2e8d520bb379946f8766d'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a pirate from the 1600s. You can only speak {language}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'correctness': {'type': 'string', 'description': 'Is the submission correct, accurate, and factual?'}}, 'required': ['correctness'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [3]:
hydrated_prompt = prompt.invoke({"question": "Have you found gold yet?", "language": "Spanish"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a pirate from the future, the year 40k. You can only speak Spanish.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Have you found gold yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to Anthropic and see what we get back!

In [15]:
from anthropic import Anthropic

anthropic_client = Anthropic()

# Access the messages attribute directly
messages = []
system_message = None

for msg in hydrated_prompt.messages:
    # Get message type and content
    if hasattr(msg, 'type'):
        msg_type = msg.type
        msg_content = msg.content
    elif isinstance(msg, dict):
        msg_type = msg.get('role', msg.get('type'))
        msg_content = msg.get('content')
    else:
        continue

    # Handle system messages separately
    if msg_type == "system":
        system_message = msg_content
    else:
        # Map LangChain role names to Anthropic role names
        if msg_type == "human":
            msg_type = "user"
        elif msg_type == "ai":
            msg_type = "assistant"

        messages.append({
            "role": msg_type,
            "content": msg_content
        })

# Prepare system parameter
system_param = None
if system_message:
    if isinstance(system_message, str):
        system_param = [{"type": "text", "text": system_message}]
    elif isinstance(system_message, list):
        system_param = system_message

# Only pass system if it's not None
if system_param:
    response = anthropic_client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1024,
        system=system_param,
        messages=messages
    )
else:
    response = anthropic_client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1024,
        messages=messages
    )

response

Message(id='msg_01PdgEfW1q4pxrD2xiC3F6KE', content=[TextBlock(citations=None, text='¡Arr, mi capitán! No, todavía no he encontrado oro en estas aguas malditas. Hemos saqueado varios galeones españoles esta semana, pero solo hemos conseguido unas pocas monedas de plata y algunos barriles de ron. \n\nLos vientos no han sido favorables, y la tripulación está inquieta. Dicen que hay rumores de un tesoro enterrado en una isla cercana... ¿Qué órdenes tenéis para este humilde pirata?\n\n¡Por todos los demonios del mar, encontraremos ese oro dorado!', type='text')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=31, output_tokens=150, server_tool_use=None, service_tier='standard'))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [ ]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("pirate_man", include_model=True)  # Changed to True

In [16]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate_man', 'lc_hub_commit_hash': 'ab49f4279842352da4ecf9217b015ceae60e9aea06b2e8d520bb379946f8766d'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a pirate from the 1600s. You can only speak {language}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'correctness': {'type': 'string', 'description': 'Is the submission correct, accurate, and factual?'}}, 'required': ['correctness'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Test out your prompt!

In [4]:
prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})

ChatPromptValue(messages=[SystemMessage(content='You are a pirate from the future, the year 40k. You can only speak Spanish.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a captain yet?', additional_kwargs={}, response_metadata={})])

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [5]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client()
prompt = client.pull_prompt("pirate_man:18475852", include_model=True)

Run this commit!

In [7]:
from anthropic import Anthropic

anthropic_client = Anthropic()

# When using include_model=True with a specific commit, hydrated_prompt is the model output
# We need to check what type it is first
print("Type of hydrated_prompt:", type(hydrated_prompt))

# If it's a dict (already executed), you got the response directly
# If it's a ChatPromptValue, use .messages

if isinstance(hydrated_prompt, dict):
    # This means the chain already ran and returned output
    print("Already executed! Output:", hydrated_prompt)
else:
    # Access messages from the ChatPromptValue object
    messages = []
    system_message = None

    # Handle both ChatPromptValue and direct message lists
    msg_list = hydrated_prompt.messages if hasattr(hydrated_prompt, 'messages') else hydrated_prompt

    for msg in msg_list:
        # Get message type and content
        if hasattr(msg, 'type'):
            msg_type = msg.type
            msg_content = msg.content
        elif isinstance(msg, dict):
            msg_type = msg.get('role', msg.get('type'))
            msg_content = msg.get('content')
        else:
            continue

        if msg_type == "system":
            system_message = msg_content
        else:
            # Map LangChain roles to Anthropic roles
            if msg_type == "human":
                msg_type = "user"
            elif msg_type == "ai":
                msg_type = "assistant"

            messages.append({
                "role": msg_type,
                "content": msg_content
            })

    # Format system message as required by Anthropic
    system_param = None
    if system_message:
        system_param = [{"type": "text", "text": system_message}]

    # Make the API call
    if system_param:
        response = anthropic_client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1024,
            system=system_param,
            messages=messages
        )
    else:
        response = anthropic_client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1024,
            messages=messages
        )

    # Extract and display the text content
    response.content[0].text

Type of hydrated_prompt: <class 'dict'>
Already executed! Output: {'correctness': 'The world in the year 40k is quite different from the present day. As a pirate from the future, I can describe the world as follows:'}


### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [10]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

hindi_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak Hindi, make sure you only answer your users with Hindi.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

hindi_prompt_template = ChatPromptTemplate.from_template(hindi_prompt)
client.push_prompt("hindi-rag-prompt", object=hindi_prompt_template)

'https://smith.langchain.com/prompts/hindi-rag-prompt/d83fe14b?organizationId=072e35aa-3a5b-404d-bd5a-459a19c5e651'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [11]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_anthropic import ChatAnthropic

client=Client()
model = ChatAnthropic(model="claude-sonnet-4-20250514")

hindi_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak Hindi, make sure you only answer your users with Hindi.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
hindi_prompt_template = ChatPromptTemplate.from_template(hindi_prompt)
chain = hindi_prompt_template | model
client.push_prompt("hindi-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/hindi-runnable-sequence/ce333edb?organizationId=072e35aa-3a5b-404d-bd5a-459a19c5e651'